# model validation 과 ensemble

이 강의에서는 여러 모델을 validation하고, 각 모델의 예측을 ensemble하여 성능을 향상시키는 방법에 대해 다룹니다. 검증 데이터를 전처리하고, 개별 모델의 평가 지표를 계산한 후, 여러 모델의 예측을 결합하여 최종 앙상블 결과를 도출하는 과정까지 설명할 것입니다. 최종 목표는 동일한 데이터셋에 대해 각 모델을 검증하고, 이들의 예측을 모아서 성능을 높이는 것입니다.

In [1]:
import os
# 상위 폴더로 이동
os.chdir('..')

In [2]:
import glob
from data_presets import ClassificationPresetEval
from torchvision.transforms.functional import InterpolationMode
from datasets import load_from_disk
from image_utils import CustomImagePipeline, run_model, get_metric_from_df
import pandas as pd
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## config와 모델 경로 설정
pretrain된 모델들이 모여있는 폴더와 dataset의 경로를 설정해줍니다.

In [3]:
folder_paths = [
    "./cifar100_facebook_convnext-small-224",
]

In [4]:
# config parameter 설정
config = {
    "dataset_dir" : '/workspace/model/hugginface_image_classifcation/data/hfdataset/test',
}

In [5]:
hf_dirs = []
for folder in folder_paths:
    hf_dirs.extend(glob.glob(f"{folder}/*/best.hf"))

## 데이터셋 준비

우리는 먼저 검증에 사용할 데이터셋을 로드합니다. 이 데이터셋은 Hugging Face의 `load_from_disk` 포맷으로 저장되어 있으며, 검증 데이터셋을 로드한 후 라벨을 가져옵니다. 또한, 데이터셋에 있는 라벨 이름과 ID 간의 매핑을 생성합니다. 이는 이후 모델 예측과 실제 라벨을 비교하는 데 필요합니다.


In [6]:
dataset = load_from_disk(config['dataset_dir'])
labels = dataset['test'].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [7]:
def preprocess_valid(batch, valid_transforms):
    """Apply train_transforms across a batch."""
    if "image" in batch:
        batch["pixel_values"] = [
            valid_transforms(image.convert("RGB")) for image in batch["image"]
    ]
    return batch

valid_transform = ClassificationPresetEval(
            crop_size=224,
            resize_size=256,
            interpolation=InterpolationMode.BILINEAR,
            use_v2=True,
        )

In [8]:
valid_ds = dataset['test']
valid_ds.set_transform(lambda batch: preprocess_valid(batch, valid_transform))

## 모델 검증

모델 검증 단계에서는 사전 학습된 모델을 로드하고, 각 모델에 대해 데이터를 입력하여 예측 결과를 도출합니다. 이를 통해 모델의 성능을 평가할 수 있는 여러 지표를 계산합니다. 이 강의에서는 각 모델의 정확도, 혼동 행렬 등의 평가 지표를 수집할 것입니다.

In [9]:
def get_metric(model_dir, dataset, id2label):
    answer_df = run_model(model_dir, dataset, id2label)
    metric_dict = get_metric_from_df(answer_df)
    return metric_dict, answer_df

## 평가 지표 수집

모든 모델의 성능을 평가한 후, 각 모델의 평가 지표를 데이터프레임으로 변환하여 비교 분석할 수 있습니다. 이를 통해 모델 간의 성능 차이를 쉽게 파악할 수 있으며, 어떤 모델이 가장 우수한지 평가할 수 있습니다.

In [10]:
metrics_list = []
answer_dfs = []
for model_dir in hf_dirs:
    metric_dict, answer_df = get_metric(model_dir, valid_ds, id2label)
    metric_dict['model_dir'] = model_dir
    metrics_list.append(metric_dict)
    answer_dfs.append(answer_df)

# Convert the collected metrics into a DataFrame
metrics_df = pd.DataFrame(metrics_list)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
metrics_df

,accuracy,top2_accuracy,precision,recall,f1_score,roc_auc,specificity,model_dir
0,0.8424,0.9314,0.8424,0.8424,0.8424,0.998036,0.998408,./cifar100_facebook_convnext-small-224/convnex...
1,0.8604,0.9382,0.8604,0.8604,0.8604,0.998401,0.998590,./cifar100_facebook_convnext-small-224/convnex...
2,0.8625,0.9387,0.8625,0.8625,0.8625,0.998392,0.998611,./cifar100_facebook_convnext-small-224/convnex...


In [12]:
for df in answer_dfs:
    df['idx_dir'] = valid_ds['dirs']
    df = df.drop('encoded_label', axis=1, inplace=True)

## 앙상블 방법

앙상블은 여러 모델의 예측을 결합하여 최종 예측을 생성하는 방법입니다. 여기서는 두 가지 앙상블 방식을 사용합니다: **소프트 보팅**(soft voting)과 **하드 보팅**(hard voting). 소프트 보팅은 각 모델의 확률 예측 값을 평균 내어 최종 클래스를 선택하는 방식이고, 하드 보팅은 각 모델이 예측한 클래스 중 다수결을 통해 최종 클래스를 선택하는 방식입니다.

- **소프트 보팅**: 모델들이 예측한 클래스 확률 값의 평균을 내고, 가장 높은 확률을 가진 클래스를 최종 예측으로 선택합니다. 이 방법은 모델들이 예측할 때 자신 없는 클래스를 잘 고려할 수 있다는 장점이 있습니다.
- **하드 보팅**: 모델들이 예측한 클래스 중 가장 많이 나온 클래스를 최종 예측으로 선택합니다. 각 모델의 예측 결과를 다수결로 결정하는 방식입니다.


In [41]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode

def ensemble_prediction_fixed(dfs, top_k=1, voting='soft'):
    # 여러 모델에서 나온 softmax 예측값을 결합
    
    label_encoding = dfs[0].iloc[:, :-2].columns
    # 실제 라벨 추출
    true_labels = dfs[0]['label']
    
    # LabelEncoder 초기화 및 실제 라벨에 맞춰 학습
    le = LabelEncoder()
    le.fit(label_encoding)
    
    # 실제 라벨을 인코딩
    encoded_true_labels = le.transform(true_labels)
    
    # 예측 DataFrame에서 LabelEncoder 클래스 순서에 맞춰 열 정렬
    prob_columns = le.classes_
    prob_preds = []
    for file in dfs:
        # 'label' 및 'idx_dir' 열 삭제
        prob_df = file.drop(columns=['label', 'idx_dir'])
        # 열을 정렬
        prob_df = prob_df[prob_columns]
        prob_preds.append(prob_df)
    
    # prob_preds는 이제 le.classes_ 순서로 정렬된 DataFrame 리스트
    if voting == 'soft':
        # 소프트 보팅: 확률 평균
        ensemble_softmax = np.mean([pred.values for pred in prob_preds], axis=0)
        
        # ensemble_softmax에서 예측된 클래스 인덱스 가져오기
        ensemble_predictions_indices = np.argmax(ensemble_softmax, axis=1)
        
        # top-k 정확도 수동 계산
        # top-k 예측 인덱스 가져오기
        topk_preds_indices = np.argsort(-ensemble_softmax, axis=1)[:, :top_k]
        # 실제 라벨이 top-k 예측 중 하나인지 확인
        topk_correct = [encoded_true_labels[i] in topk_preds_indices[i] for i in range(len(encoded_true_labels))]
        acc_topk = np.mean(topk_correct)
    
    elif voting == 'hard':
        # 하드 보팅: 각 모델의 예측 클래스 인덱스 가져오기
        predictions_indices = [np.argmax(pred.values, axis=1) for pred in prob_preds]
        
        # 예측 인덱스를 (n_samples, n_models) 형태로 변환
        predictions_indices = np.array(predictions_indices).T  # (n_samples, n_models)
        
        # 각 샘플에 대해 최빈값 계산
        ensemble_predictions_indices, _ = mode(predictions_indices, axis=1)
        
        # 결과를 1차원 배열로 변환
        ensemble_predictions_indices = ensemble_predictions_indices.flatten()
        
        # top-k 정확도는 하드 보팅에서는 의미가 없어 생략하거나 다른 방식으로 처리 가능
        acc_topk = None  # 하드 보팅에서는 top-k 정확도 의미가 없음
        
    else:
        raise ValueError("voting은 'soft' 또는 'hard'이어야 합니다.")
    
    # 예측된 인덱스를 원래 클래스 라벨로 변환
    ensemble_predictions = le.inverse_transform(ensemble_predictions_indices)
    
    # 정확도 계산
    acc = accuracy_score(encoded_true_labels, ensemble_predictions_indices)
    
    # 혼동 행렬 계산
    conf_matrix = confusion_matrix(encoded_true_labels, ensemble_predictions_indices)
    
    # 앙상블 데이터프레임 생성 (예측 클래스 포함)
    ensemble_df = pd.DataFrame({
        'ensemble_prediction': ensemble_predictions,
        'true_label': true_labels,
        'idx_dir': dfs[0]['idx_dir']
    })
    
    return acc, acc_topk, conf_matrix, ensemble_df

## 앙상블 성능 평가

앙상블을 통해 얻은 예측 결과에 대해 정확도와 혼동 행렬을 계산합니다. 이를 통해 앙상블의 성능이 개별 모델보다 얼마나 향상되었는지 확인할 수 있습니다. 또한, Top-k 정확도도 함께 계산하여, 실제 라벨이 상위 k개의 예측 중 하나에 포함되는지를 평가할 수 있습니다.

이 과정에서 우리는 소프트 보팅과 하드 보팅 방식을 비교하여, 각 방식이 어떤 상황에서 더 적합한지 설명하고, 최종적으로 더 나은 성능을 제공하는 앙상블 방식을 선택할 수 있습니다.


In [42]:
# Run the function with soft voting and top_k=3 as an example
acc, acc_topk, conf_matrix, ensemble_df = ensemble_prediction_fixed(answer_dfs, top_k=3, voting='soft')

In [43]:
acc, acc_topk

(0.8806, 0.9707)

In [44]:
acc, acc_topk, conf_matrix, ensemble_df = ensemble_prediction_fixed(answer_dfs, top_k=3, voting='hard')

In [46]:
acc, acc_topk

(0.8743, None)

In [45]:
ensemble_df

,ensemble_prediction,true_label,idx_dir
0,mountain,mountain,./data/cifar100_images/test/mountain/image_0.png
1,mountain,mountain,./data/cifar100_images/test/mountain/image_21.png
2,mountain,mountain,./data/cifar100_images/test/mountain/image_42.png
3,mountain,mountain,./data/cifar100_images/test/mountain/image_212...
4,mountain,mountain,./data/cifar100_images/test/mountain/image_397...
...,...,...,...
9995,tiger,tiger,./data/cifar100_images/test/tiger/image_8975.png
9996,tiger,tiger,./data/cifar100_images/test/tiger/image_9002.png
9997,tiger,tiger,./data/cifar100_images/test/tiger/image_9079.png
9998,tiger,tiger,./data/cifar100_images/test/tiger/image_9490.png
